In [ ]:
# First, let's set up our FastMCP server with necessary imports
from mcp.server.fastmcp import FastMCP
import mcp.types as types
from pydantic import BaseModel, Field
from typing import Dict, List, Optional
from datetime import datetime
import httpx
import asyncio
from dataclasses import dataclass
import json

# Initialize our FastMCP server
mcp = FastMCP("api_tools")

# API Configuration
@dataclass
class APIConfig:
    """API configuration"""
    base_url: str = "https://api.openweathermap.org/data/2.5"
    api_key: str = "demo_key"  # In production, use environment variables
    timeout: int = 10
    max_retries: int = 3

config = APIConfig()

# Data Models
class Location(BaseModel):
    """Location model"""
    name: str
    country: str
    latitude: float
    longitude: float

class WeatherData(BaseModel):
    """Weather data model"""
    location: Location
    temperature: float
    conditions: str
    humidity: Optional[int]
    wind_speed: float
    timestamp: datetime = Field(default_factory=datetime.now)

class ForecastDay(BaseModel):
    """Daily forecast model"""
    date: datetime
    temp_high: float
    temp_low: float
    conditions: str
    precipitation: float

class Forecast(BaseModel):
    """Forecast model"""
    location: Location
    days: List[ForecastDay]
    generated_at: datetime = Field(default_factory=datetime.now)

# System prompts
@mcp.prompt()
def system_prompt() -> str:
    """Define the API client's role"""
    return """
    You are an API integration assistant that helps users access weather data safely.
    Always validate API requests and handle rate limits and errors appropriately.
    Provide clear error messages when API calls fail.
    """

@mcp.prompt()
def error_prompt() -> str:
    """Handle API errors"""
    return """
    The API request failed.
    This could be due to:
    - Invalid API key
    - Rate limiting
    - Network issues
    - Invalid parameters
    Please check your request and try again.
    """

print("✅ FastMCP server initialized with API models and prompts!")


In [ ]:
# Let's implement our API resource provider
@mcp.resource("weather://{location}")
async def get_weather_data(location: str) -> Dict:
    """Get cached weather data for a location"""
    try:
        # In production, check cache first
        
        # Make API request
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{config.base_url}/weather",
                params={
                    "q": location,
                    "appid": config.api_key,
                    "units": "metric"
                },
                timeout=config.timeout
            )
            
            if response.status_code == 429:
                raise Exception("Rate limit exceeded")
                
            response.raise_for_status()
            data = response.json()
            
            # Parse response into our model
            weather = WeatherData(
                location=Location(
                    name=data["name"],
                    country=data["sys"]["country"],
                    latitude=data["coord"]["lat"],
                    longitude=data["coord"]["lon"]
                ),
                temperature=data["main"]["temp"],
                conditions=data["weather"][0]["description"],
                humidity=data["main"]["humidity"],
                wind_speed=data["wind"]["speed"]
            )
            
            return weather.dict()
            
    except Exception as e:
        return {"error": str(e)}

# For demo, we'll simulate the API response
async def mock_weather_api():
    return {
        "name": "San Francisco",
        "sys": {"country": "US"},
        "coord": {"lat": 37.7749, "lon": -122.4194},
        "main": {"temp": 18.5, "humidity": 65},
        "weather": [{"description": "partly cloudy"}],
        "wind": {"speed": 4.2}
    }

# Test our resource
with httpx.MockClient() as client:
    client.get = mock_weather_api  # Use our mock
    result = asyncio.run(get_weather_data("San Francisco"))
    print("🌤️ Weather Data:", json.dumps(result, indent=2))


In [ ]:
# Now let's implement our API tools
@mcp.tool()
async def get_forecast(location: str, days: int = 5) -> Dict:
    """
    Get weather forecast for a location
    
    Args:
        location: City name (e.g., 'San Francisco')
        days: Number of forecast days (1-5)
        
    Returns:
        Dictionary with forecast data
    """
    try:
        # First get current weather from resource
        weather = await get_weather_data(location)
        
        if "error" in weather:
            raise Exception(weather["error"])
            
        # Make forecast API request
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{config.base_url}/forecast",
                params={
                    "q": location,
                    "appid": config.api_key,
                    "units": "metric",
                    "cnt": days
                },
                timeout=config.timeout
            )
            
            if response.status_code == 429:
                raise Exception("Rate limit exceeded")
                
            response.raise_for_status()
            data = response.json()
            
            # Parse forecast data
            forecast_days = []
            for day in data["list"][:days]:
                forecast_days.append(
                    ForecastDay(
                        date=datetime.fromtimestamp(day["dt"]),
                        temp_high=day["main"]["temp_max"],
                        temp_low=day["main"]["temp_min"],
                        conditions=day["weather"][0]["description"],
                        precipitation=day["rain"]["3h"] if "rain" in day else 0.0
                    )
                )
                
            forecast = Forecast(
                location=Location(**weather["location"]),
                days=forecast_days
            )
            
            return {
                "success": True,
                "data": forecast.dict()
            }
            
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "prompt": "error_prompt"
        }

# For demo, we'll simulate the forecast API
async def mock_forecast_api():
    return {
        "list": [
            {
                "dt": datetime.now().timestamp(),
                "main": {"temp_max": 20.5, "temp_min": 15.2},
                "weather": [{"description": "sunny"}],
                "rain": {"3h": 0.0}
            },
            {
                "dt": (datetime.now().timestamp() + 86400),
                "main": {"temp_max": 22.0, "temp_min": 16.5},
                "weather": [{"description": "cloudy"}],
                "rain": {"3h": 2.5}
            }
        ]
    }

# Test our forecast tool
with httpx.MockClient() as client:
    client.get = mock_forecast_api  # Use our mock
    result = asyncio.run(get_forecast("San Francisco", days=2))
    print("🌡️ Forecast:", json.dumps(result, indent=2))
